In [12]:
import os
import pandas as pd

path = os.getcwd()

train_data = pd.read_csv("../../datasets/train.csv", index_col = 0, encoding = 'utf-8')

train_data.dropna(inplace = True)
train_data.drop(['감정_소분류','시스템문장1','시스템문장2','시스템문장3', '사람문장2', '사람문장3', '신체질환'], axis=1, inplace=True)
train_data.rename({'사람문장1':'text', '감정_대분류':'label'}, axis=1, inplace=True)

# from sklearn.preprocessing import LabelEncoder

# label_encoder = LabelEncoder()
# train_data['label'] = label_encoder.fit_transform(train_data['label'])

X_train = train_data['text']
y_train = train_data['label']

X_train[5:10], y_train[5:10]

(310    프리랜서로 일하면서 요즘 일이 없는데 가족들은 내가 열심히 하지 않은 탓이라고 해서...
 311     오늘 은퇴하고 왔는데 가족들이 그동안 고생했다며 나를 격려해주지 않아서 마음이 아팠어.
 312               앞으로 노후를 위해 준비할 것들이 많은데 의지할 사람은 없고 막막해.
 313         얼마 전 은행 계좌를 봤는데 생각보다 모아 둔 돈이 없더라고. 너무 충격이었어.
 314     그동안 담배를 너무 많이 피웠나 봐. 이제 와서 내가 모은 돈을 보니 너무 후회가 돼.
 Name: text, dtype: object,
 310    상처
 311    상처
 312    상처
 313    상처
 314    슬픔
 Name: label, dtype: object)

In [7]:
import torch
import numpy as np
from bert import BERT
from transformers import AutoModel, AutoTokenizer

def convert_to_tensor(corpus, tokenizer, device):
    inputs = tokenizer(corpus,
                       truncation=True,
                       return_tensors="pt",
                       max_length=50,
                       pad_to_max_length="right")
    
    embedding = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    token_type_ids = inputs['token_type_ids']
        
    inputs = {'source': torch.LongTensor(embedding).to(device),
              'token_type_ids': torch.LongTensor(token_type_ids).to(device),
              'attention_mask': attention_mask.to(device)}
    
    return inputs

# def pytorch_cos_sim(a, b):
#     return torch.nn.functional.cosine_similarity(a, b)

def pytorch_cos_sim(a, b):
    """
    Computes the cosine similarity cos_sim(a[i], b[j]) for all i and j.
    This function can be used as a faster replacement for 1-scipy.spatial.distance.cdist(a,b)
    :return: Matrix with res[i][j]  = cos_sim(a[i], b[j])
    """
    if not isinstance(a, torch.Tensor):
        a = torch.tensor(a)

    if not isinstance(b, torch.Tensor):
        b = torch.tensor(b)

    if len(a.shape) == 1:
        a = a.unsqueeze(0)

    if len(b.shape) == 1:
        b = b.unsqueeze(0)

    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]
    return torch.mm(a_norm, b_norm.transpose(0, 1))



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = BERT(AutoModel.from_pretrained('BM-K/KoSimCSE-roberta'))
tokenizer = AutoTokenizer.from_pretrained('BM-K/KoSimCSE-roberta')

model.to(device)
model.eval()

# Corpus with example sentences
corpus = ['한 남자가 음식을 먹는다.',
            '한 남자가 빵 한 조각을 먹는다.',
            '그 여자가 아이를 돌본다.',
            '한 남자가 말을 탄다.',
            '한 여자가 바이올린을 연주한다.',
            '두 남자가 수레를 숲 속으로 밀었다.',
            '한 남자가 담으로 싸인 땅에서 백마를 타고 있다.',
            '원숭이 한 마리가 드럼을 연주한다.',
            '치타 한 마리가 먹이 뒤에서 달리고 있다.']

inputs_corpus = convert_to_tensor(corpus, tokenizer, device)

corpus_embeddings = model.encode(inputs_corpus, device)

# Query sentences:
queries = ['1',
            '고릴라 의상을 입은 누군가가 드럼을 연주하고 있다.',
            '치타가 들판을 가로 질러 먹이를 쫓는다.']

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 5
for query in queries:
    query_embedding = model.encode(convert_to_tensor([query], tokenizer, device), device)

    print(query_embedding)
    break

    # cos_scores = pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    # cos_scores = cos_scores.cpu().detach().numpy()

    # top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

    # print("\n\n======================\n\n")
    # print("Query:", query)
    # print("\nTop 5 most similar sentences in corpus:")

    # for idx in top_results[0:top_k]:
    #     print(corpus[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))

tensor([[ 7.8967e-03, -5.8443e-01,  8.2720e-02, -1.0768e+00,  7.3777e-01,
         -5.1994e-01, -5.0743e-03, -1.4163e-01,  2.6563e-01, -1.0201e-01,
         -1.9327e-01, -5.3238e-01,  2.4999e-02, -5.3367e-01,  2.9292e-01,
         -2.5706e-01,  1.4655e-02,  1.1043e-01,  1.8759e-01,  8.9623e-02,
         -1.7414e-01,  1.8840e-01, -1.8659e-01,  5.6257e-01, -1.2248e-01,
          6.4579e-01,  1.8320e-01, -4.8350e-01,  3.9719e-01, -5.8897e-01,
         -2.3307e-01,  5.6784e-01, -9.3604e-01, -1.9653e-01,  1.2081e+00,
         -2.1208e-01, -2.6233e-01, -1.0585e-01, -3.6606e-01, -1.3617e-01,
          5.6181e-01,  2.6937e-01, -1.6141e-01,  5.2013e-01, -6.5507e-01,
         -1.3590e-01,  3.6612e-01,  2.2168e-01, -1.5250e-01, -9.8941e-02,
          1.7357e-01, -6.0483e-01, -4.3323e-02,  8.5040e-01,  4.3428e-01,
         -2.8558e-01,  5.0399e-02, -3.9722e-01,  9.6215e-01, -2.8339e-01,
          3.7969e-01, -4.0831e-01, -8.9945e-01, -2.0159e-01,  1.6754e-01,
         -2.1764e-01, -1.2400e-01, -4.

c:\Users\JH\miniconda3\envs\capstone\Lib\site-packages\transformers\tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("BAAI/bge-m3")
kw_model = KeyBERT(model=sentence_model)

In [1]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3',  
                       use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

sentences_1 = ["What is BGE M3?", "Defination of BM25"]
sentences_2 = ["BGE M3 is an embedding model supporting dense retrieval, lexical matching and multi-vector interaction.", 
               "BM25 is a bag-of-words retrieval function that ranks a set of documents based on the query terms appearing in each document"]

embeddings_1 = model.encode(sentences_1, 
                            batch_size=12, 
                            max_length=8192, # If you don't need such a long length, you can set a smaller value to speed up the encoding process.
                            )['dense_vecs']
embeddings_2 = model.encode(sentences_2)['dense_vecs']
similarity = embeddings_1 @ embeddings_2.T
print(similarity)
# [[0.6265, 0.3477], [0.3499, 0.678 ]]


c:\Users\JH\miniconda3\envs\capstone\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 23 files:   0%|          | 0/23 [00:00<?, ?it/s]c:\Users\JH\miniconda3\envs\capstone\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\JH\.cache\huggingface\hub\models--BAAI--bge-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python 

[[0.626  0.3474]
 [0.3499 0.678 ]]


In [27]:
model.encode(list(X_train[5:10]))

{'dense_vecs': array([[-0.0365  ,  0.02264 , -0.0603  , ..., -0.04626 , -0.04446 ,
         -0.0569  ],
        [-0.0192  ,  0.03555 , -0.04645 , ..., -0.006733, -0.011765,
         -0.03653 ],
        [-0.00999 ,  0.0509  , -0.03732 , ..., -0.02893 , -0.03296 ,
         -0.007736],
        [ 0.041   ,  0.02959 , -0.0346  , ...,  0.010056, -0.04718 ,
         -0.0789  ],
        [ 0.00789 ,  0.0373  , -0.0435  , ..., -0.0377  , -0.0432  ,
         -0.01268 ]], dtype=float16),
 'lexical_weights': None,
 'colbert_vecs': None}

In [25]:
x_train_encode = model.encode(list(X_train[5:10]))['dense_vecs']
y_train_encode = model.encode(list(y_train[5:10]))['dense_vecs']

similarity = x_train_encode @ y_train_encode.T
print(similarity)

[[0.374  0.374  0.374  0.374  0.4043]
 [0.4565 0.4565 0.4565 0.4565 0.5005]
 [0.3787 0.3787 0.3787 0.3787 0.454 ]
 [0.4043 0.4043 0.4043 0.4043 0.39  ]
 [0.3813 0.3813 0.3813 0.3813 0.4026]]
